#### 1. Explore tinkoff api

In [ ]:
!ls ../airflow/

In [ ]:
import tinvest
import pandas as pd
from configparser import ConfigParser
from datetime import datetime, timedelta

In [ ]:
config_parser = ConfigParser()

In [ ]:
config_parser.read('../airflow/tinkoff.cfg')

In [ ]:
token = config_parser.get('core', 'TOKEN_TINKOFF')

In [ ]:
client = tinvest.SyncClient(token, use_sandbox=True)

In [ ]:
client.get_portfolio()

In [ ]:
client.post_orders_limit_order(
        figi=figi,
        body=tinvest.schemas.LimitOrderRequest(
            lots=lots,
            operation=tinvest.schemas.OperationType(value=op_type),
            price=price
        )
    )

In [ ]:
client.clear_sandbox_account()

In [ ]:
client.get_portfolio_currencies()

In [ ]:
client.set_sandbox_currencies_balance(
    tinvest.schemas.SandboxSetCurrencyBalanceRequest(
        balance=100000,
        currency=tinvest.schemas.SandboxCurrency('USD')
    )
)

In [ ]:
aapl = client.get_market_search_by_ticker('AAPL')

In [ ]:
aapl_figi = aapl.payload.instruments[0].figi

In [ ]:
data = client.get_market_candles(
    aapl_figi,
    datetime.now() - timedelta(days=365),
    datetime.now(),
    tinvest.schemas.CandleResolution.day,
)

In [ ]:
data.payload.candles

In [ ]:
df = pd.DataFrame(
    data=(
        (
            candle.time,
            candle.o,
            candle.h,
            candle.l,
            candle.c,
            candle.v,
        ) for candle in data.payload.candles
    ),
    columns=(
        'time',
        'open',
        'high',
        'low',
        'close',
        'volume',
    )
)

In [ ]:
df.head()

In [ ]:
df.tail()

#### 2. Save to DB

In [ ]:
import psycopg2
from io import StringIO
import csv

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.loc[:, 'open':'close'] = df.loc[:, 'open':'close'].astype('float16')

In [ ]:
df.info()

In [ ]:
login = 'trader'
password = 'income'
host = 'localhost'
port = 5432
dbname = 'stocks'

In [ ]:
DB_URL = f'user={login} password={password} host={host} port={port} dbname={dbname}'

In [ ]:
DB_URL

In [ ]:
def load_df_to_db(df: pd.DataFrame, table_name: str) -> None:
    buffer = StringIO()
    df.to_csv(buffer, index=False, sep='|', na_rep='NUL', quoting=csv.QUOTE_MINIMAL,
             header=False, float_format='%.8f', doublequote=False, escapechar='\\')
    buffer.seek(0)
    copy_query = f"""
        COPY {table_name}({','.join(df.columns)})
        FROM STDIN
        DELIMITER '|'
        NULL 'NUL'
    """
    conn = psycopg2.connect(dsn=DB_URL)
    with conn.cursor() as cursor:
        cursor.copy_expert(copy_query, buffer)
    conn.commit()
    conn.close()

In [ ]:
load_df_to_db(df, 'aapl')